In [1]:
import pandas as pd
import hashlib
import re

# Hashing walk-through notebook

## This notebook provides a hands-on demonstration of the hashing process using synthetic data.

## We will show some examples of how things can go wrong in the hashing process, and introduce the steps that CPL takes to minimize this.

# Step 1: Inspect the data

### We will use two separate datasets to simulate an actual hashed linkage. We will not be able to observe the original data after hashing, but the ultimate goal is to identify which observations represent the same person across datasets.

### Before hashing, we need to understand how the data are organized and what we expect the underlying values to represent.

### We will start with some exploratory data analysis to help us determine what cleaning steps we should take.

### Printing some examples of the data is often a good place to start. It can't tell us everything about the dataset, but it can provide a good jumping off point.

### Let's take a look at our first dataset.

In [2]:
A = pd.read_csv("Link_testA.csv", dtype=str)
A

,first_name,last_name,street_num,address_1,address_2,suburb,postcode,state,DOB,SSN
0,nicholas,clarke,13,gaylard place,tryphinia view,wetherill park,2810,nsw,19041223,392-7795
1,mya,maksim,41,lambrigg street,brest hill,st kilda,6050,vic,19650630,148-8918
2,katelyn,campbell,11,ogden close,willandra village,preston,2148,tas,19291114,404-3690
3,makayla,herbert,5,plummer street,tarwyn park,hopetoun,3925,vic,19111103,175-6031
4,ben,armiento,17,burnell place,john flynn hospital,fairlight,3630,qld,,851-1174
...,...,...,...,...,...,...,...,...,...,...
2211,samuel,miles,138,hawkesbury crescent,,north ward,2148,sa,,276-3477
2212,jack,bridgland,170,thermeda place,mirreanda,berkeley vale,2286,nsw,19980721,293-0436
2213,lucas,portlock,2,mccaw place,greenwood,ascot vale,2680,nsw,19960325,723-7519
2214,tess,battisson,1,wyola place,malsah,west wollongong,2210,nsw,19230918,315-2618


### Now let's take a lookt at the second dataset.

In [3]:
B = pd.read_csv("Link_testB.csv", dtype=str)
B

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,phoebe,,33,kumm place,the manor garden,harris park,2323,vic,19061113,3155906
1,angip,wimmre,,baracchi crescent,mawarra herefords,marsden,2406,sa,19700928,3795146
2,blakeston,lock,59,bonnor close,,ment one,7140,wa,19590930,6335558
3,ben,grub,9,girrahween street,loma,brunswick west,1382,nsw,19360711,6198116
4,elle,mattehws,54,maddock place,glenburnie lodge,park orhcards,6058,vic,19960094,9253952
...,...,...,...,...,...,...,...,...,...,...
2195,xavier,mogavero,10,epenarra close,ramuda,west wolongong,3148,vic,19121210,2534734
2196,morrison,sophie,11,waller crescent,hmas cerberus,loganlea,3175,vic,19420302,7721195
2197,bailey,stanley,30,colmer street,roclkea,rowew bay,850,nsw,19010804,3200988
2198,caitlin,white,174,collyburlc rescent,,kurwongbah,2456,nsw,19431207,2627787


### Both datasets contain first name, last name, address, date of birth, and social security. These can all be used as part of our linkage strategy.


### First, we will focus on social security number. Let's see how it's represented in each dataset.

### Print some examples - dataset A

In [4]:
A['SSN']

0       392-7795
1       148-8918
2       404-3690
3       175-6031
4       851-1174
          ...   
2211    276-3477
2212    293-0436
2213    723-7519
2214    315-2618
2215    501-3227
Name: SSN, Length: 2216, dtype: object

### Print some examples - dataset B

In [5]:
# Print some example - dataset B
B['soc_sec_id']

0       3155906
1       3795146
2       6335558
3       6198116
4       9253952
         ...   
2195    2534734
2196    7721195
2197    3200988
2198    2627787
2199    9397671
Name: soc_sec_id, Length: 2200, dtype: object

## Looks like the data are formatted slightly differently between the two dasets - dataset A includes dashes and dataset B does not.

## What happens if we hash the values as they are right now?

In [6]:
A['SSN_hash'] = A['SSN'].apply(lambda s: hashlib.sha256(str.encode(s)).hexdigest().upper())

In [7]:
B['SSN_hash'] = B['soc_sec_id'].apply(lambda s: hashlib.sha256(str.encode(s)).hexdigest().upper())

## Now that we've applied the hashing algorithm, let's examine some specific examples more closely

In [8]:
print('Example from B:')
B.iloc[[1114]]

Example from B:


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,SSN_hash
1114,makayla,herbert,5,plummer street,tarwyn aprk,hopetoun,3925,vic,19111103,1756031,D98CDBDE61D7598A885807D667C383FA278766443CA67F...


In [9]:
print('Example from A:')
A.iloc[[3]]

Example from A:


,first_name,last_name,street_num,address_1,address_2,suburb,postcode,state,DOB,SSN,SSN_hash
3,makayla,herbert,5,plummer street,tarwyn park,hopetoun,3925,vic,19111103,175-6031,216E318274C37179DABC5B48CBFE7FAC7019894154CBDE...


### Look at the original social security numbers from the above examples. Are they the same?

### The two social security numbers differ from each other by only one string, the dash. However, the hashed values bear no relationship to each other. Once the data has been hashed, there is no way to see how close the original values were to each other.

### It can be easy to miss out on true matches for this reason. This can be especially tricky with administrative datasets, where we expect to see some variation (name changes, address changes, typos, etc.)

### Broadly, we take two approaches to handle this at CPL: data cleaning, and fuzzy matches. Let's start with the data cleaning.

# Step 2: Data cleaning

### Our initial attempt at hashing contains unclean data, we'll remove these and re-hash when we're more confident that the values in both datasets align.

In [10]:
A = A.drop(columns=['SSN_hash'])
B = B.drop(columns = ['SSN_hash'])

### The dashes don't contain any additional information - all we really care about are the underlying numbers. As a first step, let's remove dashes in dataset A.

In [11]:
A['SSN'] = A['SSN'].str.replace('-', '')

### Since we can't manually examine every observation in the dataset, we will conduct some additional checks to identify other differences we can't see at first glance.

### We identified that there are dashes in dataset A - what about other non-numeric characters?

In [12]:
A['find_non_numeric'] = A['SSN'].apply(lambda x: re.search(r'[^0-9]', x).end() if re.search(r'[^0-9]', x) is not None else None)

A.groupby('find_non_numeric', dropna=False).size()

find_non_numeric
1.0      16
NaN    2200
dtype: int64

### Let's check for this in dataset B as well

In [13]:
B['find_non_numeric'] = B['soc_sec_id'].apply(lambda x: re.search(r'[^0-9]', x).end() if re.search(r'[^0-9]', x) is not None else None)

B.groupby('find_non_numeric', dropna=False).size()

find_non_numeric
NaN    2200
dtype: int64

### Dataset B has all-numeric SSNs, but there are 16 observations with non-numeric characters in dataset A - let's take a closer look.

In [14]:
A[A['find_non_numeric'].notna()]

,first_name,last_name,street_num,address_1,address_2,suburb,postcode,state,DOB,SSN,find_non_numeric
116,caitlin,weaver,298,vagabond crescent,francis chambers,thornbury,5072,nsw,19680909,nan,1.0
229,angelina,campbell,161,jackie howe crescent,bugoren,woorim,6052,nsw,19531108,nan,1.0
249,harrison,burford,30,de salis street,gwandalan,molong,4105,vic,19511229,nan,1.0
287,joshua,golden,22,mcintosh street,ra 23012,noble park,2753,sa,19940329,nan,1.0
523,evan,kranz,33,pandanus street,tantallon,bowral,2264,nsw,19330726,nan,1.0
925,,vincent,47,kent street,blue hills,forest hill,4655,vic,19190304,nan,1.0
1017,benjamin,yen,43,kambalda crescent,elmwood,findon,3807,nsw,19850706,nan,1.0
1082,pino,shepherd,55,warragamba avenue,pineview farm,sale,5608,vic,19100226,nan,1.0
1454,alicia,riddei,11,elvire place,swanleigh,allawah,2440,qld,19620629,nan,1.0
1488,madison,chandler,29,luckman place,hd tickera,pearl beach,2133,nsw,19551128,nan,1.0


### These are all missing values that got coded as the string literal 'nan'!

### Note that the hashed value would be the same for all of these observations. However, we don't really want to conclude that these people all have the same SSN because we don't know the underlying value.

### We can handle this by creating a flag that will mark cases that don't meet our criteria for this field.

### For SSN, we expect everything to be numeric - so things like dashes, letters, etc. are first removed.

### After removing non-numeric characters, we check for the following:

- Longer than 9 characters
- If shorter than 9 characters, we pad the left side with 0s in case any leading 0s were dropped
- Starts with "9", "000", or "666"
- Ends with "0000"
- Is a known bad SSN such as "123456789"
- The middle two digits are "00" (eg "123-00-6789")

### Any SSN that meets any of the above criteria is marked as "bad" and is not hashed. 
### This helps save space (the hashed values are large!) while safeguarding against accidental matches.
### First, remove non-numeric characters and pad with leading 0s where necessary. If we add too many 0s that's ok - these will be caught in later checks.

In [15]:
A['SSN'] = A['SSN'].str.replace(r'[^0-9]', '', regex=True)
B['soc_sec_id'] = B['soc_sec_id'].str.replace(r'[^0-9]', '', regex=True)

A['SSN'] = A['SSN'].str.pad(width=7, side='left', fillchar='0')
B['soc_sec_id'] = B['soc_sec_id'].str.pad(width=7, side='left', fillchar='0')

### Now we can identify 'bad' SSNs.
### Note that our dummy data in this example has fake SSNs that are only 7 digits, but for real data we would expect social security numbers to be 9 digits long.

In [16]:
def create_bad_flag(df, ssn_col_name):
    df['badSSN'] = (
            (df[ssn_col_name] == '') |
            (df[ssn_col_name].str.len() > 7) |
            (df[ssn_col_name].str[0:3].isin(["000", "666"])) |
            (df[ssn_col_name].str[3:5]=="00") |
            (df[ssn_col_name].str[5:7]=="00") |
            (df[ssn_col_name].str.startswith('9')) |
            (df[ssn_col_name].isin(["078051120", "123456789"]))
        )
    return df

A = create_bad_flag(A, 'SSN')
B = create_bad_flag(B, 'soc_sec_id')

### How many bad SSNs do we have?

In [17]:
A.groupby(['badSSN']).size()

badSSN
False    1915
True      301
dtype: int64

In [18]:
B.groupby(['badSSN']).size()

badSSN
False    1928
True      272
dtype: int64

### In this section, we examined the data to understand what cleaning steps we needed to take before hashing.  However, in most cases we don't have access to the data before it is hashed.

### We rely on interviews with partners to understand how their data are stored. These interviews can reveal things like the presence of the dashes in dataset A, and prompt us to add certain cleaning steps. Additionally, it is good practice to maintain a set of common cleaning steps that will be taken for all projects to catch anomalies in the data. Even if we hadn't realized that missing values in dataset A were being captured as the string 'nan', this would have been fixed because we removed non-numeric characters regardless.

# Step 3: Prepping for fuzzy matches (substrings)

### The term "fuzzy match" refers to a pairing between values that may not exactly match, but are close enough to be considered the same. For example, the names "Sarah" and "Sara" could be considered a fuzzy match.

### We expect people's PII to change over time (e.g. name changes or address changes). Furthermore, there is the possibility for typos or other mistakes in the data.

### When linking administrative datasets, researchers will often allow for fuzzy matches to account for this variation in PII. One common way to approach fuzzy matches is to consider string distances: for a given pair of strings, what is the minimal number of string additions or deletions one would have to make in order to go from A to B? For example, "Sarah" and "Sara" have a string distance of 1.

### With hashed data, we are unable to look at string distances directly. Instead, we take smaller substrings of the original values and hash these instead.

### Let's look an example of what a "fuzzy" match might look like.

In [19]:
B[B['soc_sec_id']=='7803602']

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,find_non_numeric,badSSN
1100,victoria,zimmermann,37,o'sullivan street,kelgoola,nyah,2842,sa,19240517,7803602,None,False


In [20]:
A[A['SSN'] == '7803601']

,first_name,last_name,street_num,address_1,address_2,suburb,postcode,state,DOB,SSN,find_non_numeric,badSSN
236,victoria,zimmermann,37,o'sullivan street,kelgoola,nyah,2842,qld,19240517,7803601,NaN,False


### Since the above examples have identical PII except for the SSN, and the SSN is only off by one digit, it seems likely that these are the same person. We will hash several substrings of the SSN so that we can capture this similarity post-hashing

### For SSN, we remove every pair of two consecutive digits, and hash the remainder

In [21]:
for i in range(1, 7):
            A.loc[~A['badSSN'], 'ssn' + str(i) + str(i+1)] = A['SSN'].str[0:i-1] + A['SSN'].str[i+1:9]


In [22]:
for i in range(1, 7):
            B.loc[~B['badSSN'], 'ssn' + str(i) + str(i+1)] = B['soc_sec_id'].str[0:i-1] + B['soc_sec_id'].str[i+1:9]


In [23]:
B[B['soc_sec_id']=='7803602']

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,find_non_numeric,badSSN,ssn12,ssn23,ssn34,ssn45,ssn56,ssn67
1100,victoria,zimmermann,37,o'sullivan street,kelgoola,nyah,2842,sa,19240517,7803602,None,False,03602,73602,78602,78002,78032,78036


In [24]:
A[A['SSN'] == '7803601']

,first_name,last_name,street_num,address_1,address_2,suburb,postcode,state,DOB,SSN,find_non_numeric,badSSN,ssn12,ssn23,ssn34,ssn45,ssn56,ssn67
236,victoria,zimmermann,37,o'sullivan street,kelgoola,nyah,2842,qld,19240517,7803601,NaN,False,03601,73601,78601,78001,78031,78036
